# Greedy Search Algorithm

<h3>Algorithm Implementation</h3>

In [3]:
from graph import Graph

def greedy_search(graph: Graph, start: str, goal: str) -> bool:
    visited = set([start])

    curr_city = start
    unvisited_neighbours = [(neighbour, weight) for neighbour, weight in graph.get_neighbours(curr_city)]
    path = [start]

    while unvisited_neighbours:
        for city, weight in unvisited_neighbours:
            if city == goal:
                path.append(city)
                return path
        
        curr_city = min(unvisited_neighbours, key=lambda x: x[1])[0]
        visited.add(curr_city)
        path.append(curr_city)

        unvisited_neighbours = [(neighbour, weight) for neighbour, weight in graph.get_neighbours(curr_city) if neighbour not in visited]

    return

##### A. The benchmark should be finding the path between each node. Randomly pick 10 cities. Find the path between them.

In [ ]:
import timeit
from random import shuffle
from graph import load_romania_graph

romania: Graph = load_romania_graph()

romanian_cities: list[str] = [city for city in romania.get_nodes()]
shuffle(romanian_cities)

random_cities: list[str] = romanian_cities[:10]

for city1 in random_cities:
    for city2 in random_cities:
        if city1 is not city2:
            path = greedy_search(graph=romania, start=city1, goal=city2)
            time_taken = timeit.timeit(lambda: greedy_search(romania, city1, city2), number=10) # number=10 means the code will be run 10 tiems and the average will be returned

            print(f"Trying: {city1} -> {city2}")
            if path:
                print(f"Found path: {' -> '.join(path)}, Length: {len(path)}")
            else:
                print("Cannot find path.")
            print(time_taken, "seconds")

##### D. Create random graphs with a number of nodes n = 10, 20, 30, 40. Randomly connect nodes with the probability of edges p = 0.2, 0.4, 0.6, 0.8. In total, you will have 16 graphs.

In [5]:
import random

def create_random_graph(num_nodes: int, edge_prob: float) -> Graph:
    graph = Graph()
    
    # add all nodes to the graph
    for i in range(num_nodes):
        graph.add_node(str(i))
        
    # randomly connect nodes with probability edge_prob
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            if random.random() < edge_prob:
                weight = random.randint(1, 10)
                graph.insert_edge(str(i), str(j), weight)
    
    return graph


In [51]:
from random import shuffle

node_numbers = [10, 20, 30, 40]
edge_probabilities = [ 0.2, 0.4, 0.6, 0.8]

experiment_graphs = []

for num_nodes in node_numbers:
    for edge_prob in edge_probabilities:
        experiment_graphs.append((f"Number of nodes: {num_nodes}, Probability of Edges: {edge_prob}", create_random_graph(num_nodes, edge_prob)))

results = {}

for label, graph in experiment_graphs:
    nodes = graph.get_nodes()

    shuffle(nodes)

    random_5_nodes = nodes[:5]

    results[label] = {}

    for node1 in random_5_nodes:
        for node2 in random_5_nodes:
            if node1 is not node2:
                path = greedy_search(graph, node1, node2)
                time_taken = timeit.timeit(lambda: greedy_search(graph, node1, node2), number=5)

                results[label][f"[{node1} -> {node2}]"] = (time_taken * 1000, path)

In [ ]:
import matplotlib.pyplot as plt

# Define the plot titles for each subplot pair
plot_titles = ['Time taken vs Route Search', 'Solution Length Vs Route Search']

for i, (label, result) in enumerate(results.items()):
    search_list = list(result.keys())
    time_taken_list = list(time for time, path in result.values())
    solution_length_list = list(len(path) if path else 0 for time, path in result.values())

    fig, axs = plt.subplots(1, 2, figsize=(30, 10))

    axs[0].plot(search_list, time_taken_list)
    axs[0].set_xlabel('Route Search (from node a -> b)')
    axs[0].set_ylabel('Time taken (seconds)')
    axs[1].plot(search_list, solution_length_list)
    axs[1].set_xlabel('Route Search (from node a -> b)')
    axs[1].set_ylabel('Solution Length')

    # Set the title for each subplot pair
    for j, ax in enumerate(axs):
        ax.set_title(plot_titles[j])
        
    plt.suptitle(f'{label}', fontsize=16)
    plt.tight_layout()
    plt.show()